In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import numpy as np
from datetime import datetime
import os
import glob
import shutil
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import yfinance as yf


In [2]:
# Appends new row to existing csv file
# path = "TEST/testing.csv"

def append_row_toCSV(path, toadd_data):
    mydata = pd.read_csv(path)
    newdata = pd.DataFrame([toadd_data], columns=mydata.columns)
    mydata = pd.concat([mydata, newdata], ignore_index=True)
    mydata.to_csv(path, index=False)



In [ ]:
Earnings beat/miss

Analyst ratings compared to current price
https://www.marketbeat.com/stocks/NASDAQ/AAPL/price-target/

In [151]:
# finds earnings estimate and and EPS 
def add_EPS_estimate_reported(url):
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, "html.parser")
    body = soup.find("table", class_="scroll-table sort-table", id="earnings-history")

    body = body.find("tbody").find_all("tr")
    body.pop(0)
    mylist = []
    for i in body:
        # print(i.find("td").contents)
        while True:
            date_EPS = i.find("td")
            consensus_EPS = date_EPS.find_next("td").find_next("td") 
            reported_EPS = consensus_EPS.find_next("td")
            if str(date_EPS.contents[0])[0] == "<":
                break
            date_EPS = date_EPS.contents
            consensus_EPS = consensus_EPS.contents
            reported_EPS = reported_EPS.contents
            mylist2 = [date_EPS,consensus_EPS,reported_EPS]
            mylist.append(mylist2)
            break
    return mylist

In [250]:
#check if the difference betweens two dates are below 5 to match with their respective quarterly report dates
#checks if the data on consensus/reported EPS match the quarterly report one
# since it is possible for them to be different as they could be released at different dates and depends on whether the websites I used
# to scrape from update it instantenously or not, thus ±5 days error range.
def check_day_diff(mydatestr, quarterURL):
    
    date_1 = datetime.strptime(mydatestr, f'%m/%d/%Y')

    myurl = pd.read_csv(quarterURL)
    date_string = myurl.loc[1][0]
    date_2 = datetime.strptime(date_string, f'%Y-%m-%d %H:%M:%S')

    day_difference = abs((date_2 - date_1).days)
    if day_difference < 5:
        return True
    else:
        return False

In [264]:
# sorts directory into a list, then add consensus and reported EPS to each stock's csv files
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        all_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
        all_files.sort(reverse=True) #gives a SORTED list of all .csv paths of that particular stock 
        url = 'https://www.marketbeat.com/stocks/NASDAQ/'+"MSFT"+'/earnings/'
        eps_list = add_EPS_estimate_reported(url)
        print(eps_list)
        i=0
        try:
            for quarterURL in all_files: #loops around each .csv path
                date = ''.join(eps_list[i][0])
                mybool = check_day_diff(date, quarterURL) #checks if the data on consensus/reported EPS match the quarterly report one
                consensus_EPS = ''.join(eps_list[i][1]).replace("$","") #gets consensus data, turning it into a string in the format i.e. $0.48 then getting rid of the $ sign --> to get 0.48
                reported_EPS = ''.join(eps_list[i][2]).replace("$","")
                
                percentage = (float(reported_EPS)/float(consensus_EPS))-1
                print(percentage)
                
                if mybool == True:
                    path = "TEST/testing.csv"
                    #path = quarterURL
                    append_row_toCSV(path, percentage) #adds percentage of surprise to csv file
                i+=1
        except:
            pass
        break
    break


[[['10/25/2022'], ['$2.29'], ['$2.35']], [['7/26/2022'], ['$2.28'], ['$2.23']], [['4/26/2022'], ['$2.18'], ['$2.22']], [['1/25/2022'], ['$2.29'], ['$2.48']], [['10/25/2021'], ['$2.08'], ['$2.27']], [['7/26/2021'], ['$1.92'], ['$2.17']], [['4/27/2021'], ['$1.76'], ['$1.95']], [['1/25/2021'], ['$1.64'], ['$2.03']], [['10/27/2020'], ['$1.53'], ['$1.82']], [['7/22/2020'], ['$1.34'], ['$1.46']], [['4/29/2020'], ['$1.27'], ['$1.40']], [['1/29/2020'], ['$1.32'], ['$1.51']], [['10/23/2019'], ['$1.24'], ['$1.38']], [['7/18/2019'], ['$1.21'], ['$1.37']], [['4/24/2019'], ['$1.00'], ['$1.1375']], [['1/30/2019'], ['$1.09'], ['$1.10']], [['10/24/2018'], ['$0.96'], ['$1.14']], [['7/19/2018'], ['$1.08'], ['$1.13']], [['4/26/2018'], ['$0.85'], ['$0.95']], [['1/31/2018'], ['$0.86'], ['$0.96']], [['10/26/2017'], ['$0.72'], ['$0.84']], [['7/20/2017'], ['$0.71'], ['$0.98']], [['4/27/2017'], ['$0.69'], ['$0.73']], [['1/26/2017'], ['$0.79'], ['$0.83']], [['10/20/2016'], ['$0.68'], ['$0.76']], [['7/19/2016'],

Used yfinance instead of scraping it myself since the combination of url of each stock on marketbeat is different
i.e. 
NASDAQ/MSFT/earnings
NYSE/JPM/earnings

also yfinance has more historical data (see below compared to above), both MSFT


In [17]:
import yfinance as yf


msft = yf.Ticker("AES")
earnings_list = msft.get_earnings_dates(limit=200)
earnings_list.reset_index(inplace=True)
earnings_list = earnings_list[earnings_list['Surprise(%)'].notna()]
eps_list = earnings_list.reset_index(drop=True)

# TEST KO



In [38]:
eps_list[0:50]

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%)
0,2022-11-04 02:00:00-04:00,0.53,0.63,0.1820
1,2022-08-04 13:00:00-04:00,0.31,0.34,0.0828
2,2022-05-05 13:00:00-04:00,0.28,0.21,-0.2391
3,2022-01-06 14:00:00-05:00,0.43,0.45,0.0369
4,2021-11-03 14:00:00-04:00,0.43,0.50,0.1521
5,2021-08-04 12:00:00-04:00,0.29,0.31,0.0727
6,2021-05-06 02:00:00-04:00,0.30,0.28,-0.0635
7,2021-02-25 01:00:00-05:00,0.44,0.48,0.0909
8,2020-11-06 01:00:00-05:00,0.43,0.42,-0.0255
9,2020-08-06 02:00:00-04:00,0.23,0.25,0.0870


In [26]:
date = eps_list['Earnings Date'][0]
date_string = eps_list['Earnings Date'][1]
date

In [36]:
percentage_surprise = float(eps_list['Surprise(%)'][0])/100
percentage_surprise

0.002504

In [37]:
#Adjusted for Yfinance
# sorts directory into a list, then add the difference between consensus and reported EPS to each stock's csv files
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    print(sector, "sector start ................................................................")
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        print(stock, "start")
        all_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
        all_files.sort(reverse=True) #gives a SORTED list of all .csv paths of that particular stock 
        try:
            yf_stock = yf.Ticker(stock)
            earnings_list = yf_stock.get_earnings_dates(limit=200) #uses yfinance and gets historical eps consensus/reported data
            earnings_list.reset_index(inplace=True)
            earnings_list = earnings_list[earnings_list['Surprise(%)'].notna()]
            eps_list = earnings_list.reset_index(drop=True)
            print(eps_list)
            number_csv = 0
            number_EPS = 0
            number_csv_2 = 0
            try:
                while number_csv < len(all_files): #loops around each .csv path
                    quarterURL = all_files[number_csv] 
                    percentage_surprise = float(eps_list['Surprise(%)'][number_EPS])/100
                    date = eps_list['Earnings Date'][number_EPS]
                    date = date.replace(tzinfo=None) #fixes "Cannot subtract tz-naive and tz-aware datetime-like objects" error
                    mybool = check_day_diff(date, quarterURL) #checks if the data on consensus/reported EPS match the quarterly report one
                    
                    read_csv = pd.read_csv(quarterURL)
                    print(read_csv)

                    if mybool == True:
                        number_csv += 1
                        number_EPS += 1
                        number_csv_2 += 1
                        if len(read_csv) > 3:
                            read_csv = read_csv[:3] # if data has failed to load (i.e. printed as NaN), the previous Nan Data will be wiped, and the process will be redone
                        newdata = pd.DataFrame([percentage_surprise], columns=read_csv.columns)
                        read_csv = pd.concat([read_csv, newdata], ignore_index=True)
                        read_csv.to_csv(quarterURL, index=False)
# number_csv_2 is used with another mybool2 to check if the EPS quarterly report date matches with the date on the .csv file, in case it was reversed
#e.g.
# case 1: uses mybool
# csv       eps
# 2020      2022 (mybool = False)
# 2020      2021 (mybool = False)
# 2020      2020 (MATCH, mybool = True)
# 
# case 2: uses mybool2
# csv       eps
# 2022      2020 (mybool = False)
# 2021      2020 (mybool = False)
# 2020      2020 (MATCH, mybool2 = True)
#

                    elif mybool == False:
                        number_EPS += 1
                        number_csv_2 += 1
                        quarterURL = all_files[number_csv_2] 
                        percentage_surprise = float(eps_list['Surprise(%)'][number_EPS])/100
                        date = eps_list['Earnings Date'][number_EPS]
                        date = date.replace(tzinfo=None) 
                        mybool2 = check_day_diff(date, quarterURL)
                        if mybool2 == True:
                            number_EPS -=1
                            read_csv = pd.read_csv(quarterURL)
                            number_csv += 1
                            number_EPS += 1
                            number_csv_2 += 1

                        if len(read_csv) < 4:
                            append_row_toCSV(quarterURL, np.NaN)
            except:
                pass

        except:
            print("ERROR", stock, "NOT FOUND!!!!!!")
            with open("not_found_list_stocks.txt", "a") as file_object:
                file_object.write(stock)
            pass
    break


utilities start
AES start
                Earnings Date  EPS Estimate  Reported EPS  Surprise(%)
0   2022-11-04 02:00:00-04:00          0.53          0.63       0.1820
1   2022-08-04 13:00:00-04:00          0.31          0.34       0.0828
2   2022-05-05 13:00:00-04:00          0.28          0.21      -0.2391
3   2022-01-06 14:00:00-05:00          0.43          0.45       0.0369
4   2021-11-03 14:00:00-04:00          0.43          0.50       0.1521
..                        ...           ...           ...          ...
113 1994-02-18 00:00:00-05:00          0.07          0.08       0.1272
114 1993-10-20 00:00:00-04:00          0.08          0.08      -0.0400
115 1993-07-14 00:00:00-04:00          0.07          0.08       0.0360
116 1993-04-20 00:00:00-04:00          0.07          0.07       0.0697
117 1993-02-10 00:00:00-05:00          0.05          0.05       0.0560

[118 rows x 4 columns]
                                                2022
0                                            

In [3]:
#Adjusted for Yfinance
#check if the difference betweens two dates are below 6 to match with their respective quarterly report dates
def check_day_diff(date_1,QuarterURL):
    myurl = pd.read_csv(QuarterURL)
    date_string = myurl.loc[1][0]
    date_2 = datetime.strptime(date_string, f'%Y-%m-%d %H:%M:%S')
    
    day_difference = abs((date_2 - date_1).days)
    if day_difference < 5:
        return True
    else:

        return False

